## Testing the Environment

In [1]:
import importlib

packages = [
    "datasets", "transformers", "accelerate", "soundfile",
    "librosa", "evaluate", "jiwer", "tensorboard", "gradio"
]

for pkg in packages:
    try:
        version = importlib.import_module(pkg).__version__
        print(f"{pkg}: ✅ Installed (version {version})")
    except ImportError:
        print(f"{pkg}: ❌ Not installed")
    except AttributeError:
        print(f"{pkg}: ✅ Installed (version unknown)")

datasets: ✅ Installed (version 3.5.0)


C:\LLM Instruction Fine-Tuning\.venv\Lib\site-packages\transformers\utils\generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


transformers: ✅ Installed (version 4.35.2)


W0424 09:13:53.203000 27580 Lib\site-packages\torch\distributed\elastic\multiprocessing\redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.
C:\LLM Instruction Fine-Tuning\.venv\Lib\site-packages\transformers\utils\generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


accelerate: ✅ Installed (version 0.24.1)
soundfile: ✅ Installed (version 0.13.1)
librosa: ✅ Installed (version 0.11.0)


C:\LLM Instruction Fine-Tuning\.venv\Lib\site-packages\transformers\utils\generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


evaluate: ✅ Installed (version 0.4.3)
jiwer: ✅ Installed (version unknown)
tensorboard: ✅ Installed (version 2.19.0)
gradio: ✅ Installed (version 5.25.0)


In [2]:
import torch
print(torch.cuda.is_available())

True


## Transcribing with whisper-large-v3 model the single long audio

## Comparison with the Whisper-Small Model

📌 **Conclusion**  

In this step, we transcribed the same Azerbaijani audio using both the **Whisper Large-v3** and **Whisper Small** models via `stable-ts`. The comparison revealed that the **Whisper Small model's output was significantly less accurate** than that of the **Large-v3** model. However, this gap presents a great opportunity: by using the high-quality transcriptions from the Large-v3 model as reference data, we can **fine-tune the Small model** to significantly improve its performance — especially for transcription tasks in under-resourced languages like Azerbaijani.

## Fine-Tuning Whisper-Small

### Chop long audio into segments using the time stamps

### Fine-Tuning

In [1]:
# 1. Check for GPU
import torch
print("🚀 Using device:", "cuda" if torch.cuda.is_available() else "cpu")

# 2. Install dependencies (uncomment if running in a fresh environment)
# !pip install -q datasets>=2.6.1 transformers>=4.30.0 accelerate librosa evaluate jiwer soundfile

# 3. Imports
import os
from datasets import load_from_disk, Audio
from transformers import (
    WhisperFeatureExtractor,
    WhisperTokenizer,
    WhisperProcessor,
    WhisperForConditionalGeneration,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
)
import evaluate
from dataclasses import dataclass
from typing import Any, Dict, List, Union
import torch

# 4. Load your dataset
ds = load_from_disk("./az_transcription_dataset_segments")
print(ds)

# 5. Cast audio column to the 16 kHz Whisper format
ds = ds.cast_column("audio", Audio(sampling_rate=16_000))

# 6. Load Whisper processor for "small" and set to Azerbaijani
feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-small")
tokenizer = WhisperTokenizer.from_pretrained(
    "openai/whisper-small",
    language="Azerbaijani",   # Whisper’s language token
    task="transcribe"
)
processor = WhisperProcessor.from_pretrained(
    "openai/whisper-small",
    language="Azerbaijani",
    task="transcribe"
)

# 7. Prepare the dataset: compute log-Mel inputs and tokenize transcripts
def prepare_batch(batch):
    # load + resample audio
    audio_arr = batch["audio"]["array"]
    sr = batch["audio"]["sampling_rate"]
    # feature extraction
    features = feature_extractor(audio_arr, sampling_rate=sr).input_features[0]
    # tokenize
    labels = tokenizer(batch["sentence"]).input_ids
    return {"input_features": features, "labels": labels}

ds = ds.map(
    prepare_batch,
    remove_columns=ds.column_names,
    num_proc=1
)

# Split into train & test sets
ds = ds.train_test_split(test_size=0.1, seed=42)
train_ds, eval_ds = ds["train"], ds["test"]

# 8. Data collator to batch and pad correctly
@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any  # the WhisperProcessor

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # separate audio inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        inputs = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(inputs, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        label_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = label_batch["input_ids"].masked_fill(label_batch.attention_mask.ne(1), -100)
        
        # remove leading bos (beginning of a sentence) if present:
        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels
        
        return batch

data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

# 9. Load WER metric
wer_metric = evaluate.load("wer")

# We need to craft a function that processes our model predictions and calculates the WER metric. 
# This function, named `compute_metrics`, initially substitutes `-100` with the `pad_token_id` in the `label_ids`, 
# reversing the adjustment made in the data collator to accurately exclude padded tokens from the loss calculation. 
# Subsequently, it translates the predicted and label ids into strings. 
# Ultimately, it determines the WER by comparing the predictions with the reference labels:
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)
    wer = wer_metric.compute(predictions=pred_str, references=label_str)
    
    return {"wer": wer}

# 10. Load the pretrained Whisper-Small model
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")
model.generation_config.language = "az" # Set decoding language

# Adjust generation parameters - no tokens are predetermined as decoder outputs, and no tokens are excluded during the generation process
model.config.forced_decoder_ids = None
model.config.suppress_tokens = []

# 11. Setup training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./whisper-az-small-finetuned",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=4,
    learning_rate=5e-5,
    warmup_steps=10,
    max_steps=500,
    evaluation_strategy="steps",
    eval_steps=100,
    save_steps=100,
    logging_steps=20,
    fp16=True,
    predict_with_generate=True,
    generation_max_length=225,
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=False,
)

# 12. Initialize Trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=eval_ds,
    data_collator=data_collator,
    tokenizer=processor.tokenizer, # repo uses "tokenizer=processor.feature_extractor", but it could be wrong!
    compute_metrics=compute_metrics,
)

# 13. Save processor (necessary for inference)
processor.save_pretrained(training_args.output_dir)

# 14. Launch training
trainer.train()

🚀 Using device: cuda


C:\LLM Instruction Fine-Tuning\.venv\Lib\site-packages\transformers\utils\generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
C:\LLM Instruction Fine-Tuning\.venv\Lib\site-packages\transformers\utils\generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
W0424 16:36:00.320000 9828 Lib\site-packages\torch\distributed\elastic\multiprocessing\redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.
C:\LLM Instruction Fine-Tuning\.venv\Lib\site-packages\transformers\utils\generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


Dataset({
    features: ['audio', 'sentence'],
    num_rows: 1618
})


C:\LLM Instruction Fine-Tuning\.venv\Lib\site-packages\huggingface_hub\file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Map:   0%|          | 0/1618 [00:00<?, ? examples/s]

C:\LLM Instruction Fine-Tuning\.venv\Lib\site-packages\huggingface_hub\file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
C:\LLM Instruction Fine-Tuning\.venv\Lib\site-packages\accelerate\accelerator.py:439: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


Step,Training Loss,Validation Loss,Wer
100,0.608200,0.725551,0.602782
200,0.159100,0.714373,0.863988
300,0.064500,0.737078,0.547141
400,0.021000,0.739818,0.528594
500,0.009600,0.750181,0.528594


There were missing keys in the checkpoint model loaded: ['proj_out.weight'].


TrainOutput(global_step=500, training_loss=0.36935254165530207, metrics={'train_runtime': 1203.1897, 'train_samples_per_second': 6.649, 'train_steps_per_second': 0.416, 'total_flos': 2.30868320256e+18, 'train_loss': 0.36935254165530207, 'epoch': 5.49})